#### Samiksya Subedi
#### Elevation data interpolation
#### This notebook creates 3 interpolation methods inelevation data and determines best interpolation methods. 

In [1]:
import arcpy
import requests
import zipfile
import os
import json
import pandas as pd
import psycopg2
from psycopg2 import sql
from io import StringIO
import arcgis
from arcgis.features import GeoAccessor, GeoSeriesAccessor

#### Connecting  to postgis and getting elevation points

In [2]:
sde_connection = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\PostgreSQL-34-gis5572(postgres).sde"
elevation = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\gis5572_points_in_sde_clip'
output_directory = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3"
output_gdb = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\Lab3.gdb"

### Creating a subset of original data

In [3]:
arcpy.conversion.FeatureClassToShapefile(
    Input_Features="gis5572_points_in_sde_Clip",
    Output_Folder=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3"
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab3'>

In [4]:
import arcpy
import numpy as np

# Set workspace environment
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3"

# Input feature class
input_features = "gis5572_points_in_sde_Clip"

# Output folder
output_folder = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3"

# Convert feature class to shapefile
arcpy.conversion.FeatureClassToShapefile(input_features, output_folder)

# Count the total number of features in the input feature class
total_count = int(arcpy.management.GetCount(input_features)[0])

# Number of features to select
num_to_select = 25  # Change this number according to requirement

# Generate random object IDs to select
rand_ids = [str(i) for i in range(1, total_count + 1)]
selected_ids = set(np.random.choice(rand_ids, num_to_select, replace=False))

# Create a query to select features with selected object IDs
query = f"OBJECTID IN ({','.join(selected_ids)})"

# Create a new feature layer with selected features
subset_feature_layer = arcpy.management.MakeFeatureLayer(input_features, "elevation_points", query)



### Interpolation

In [9]:
import arcpy

# IDW
with arcpy.EnvManager(mask="Minnesota_shapefile"):
    arcpy.ddd.Idw(
        in_point_features="elevation_points",
        z_field="pointid",
        out_raster=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb\Idw_ele",
        cell_size=2160,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )

# Ordinary Kriging
with arcpy.EnvManager(mask="Minnesota_shapefile", scratchWorkspace=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="elevation_points",
        z_field="pointid",
        kriging_model="Spherical # # # #",
        cell_size=2160,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb\Kriging_ele")

# Universal Kriging
with arcpy.EnvManager(mask="Minnesota_shapefile", scratchWorkspace=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="elevation_points",
        z_field="pointid",
        kriging_model="LinearDrift 2160.000000 # # #",
        cell_size=2160,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb\Kriging_uniele")


### Accuracy assesment

In [6]:
arcpy.ga.ExploratoryInterpolation(
    in_features="elevation_points",
    value_field="pointid",
    out_cv_table=r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb\Accuracyelevation",
    out_geostat_layer=None,
    interp_methods="SIMPLE_KRIGING;ORDINARY_KRIGING;UNIVERSAL_KRIGING",
    comparison_method="SINGLE",
    criterion="ACCURACY",
    criteria_hierarchy="ACCURACY PERCENT #",
    weighted_criteria="ACCURACY 1",
    exclusion_criteria=None
)

<Result 'C:\\Users\\samik\\OneDrive\\Documents\\ArcGIS\\Projects\\Lab3elevation\\Lab3elevation.gdb\\Accuracyelevation'>

### Push Accuracy assesment to remote database

In [15]:
import arcpy

# Set the workspace environment
arcpy.env.workspace = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\PostgreSQL-34-gis5572(postgres).sde"

# Define the output table
out_table = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb\Accuracyelevation"

# Define the output table name in the PostGIS database
table_name = "Accuracyelevation"

# Define connection parameters for PostGIS
db_connection = r"C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3\PostgreSQL-34-gis5572(postgres).sde"

# Copy the rows directly to the PostGIS database
arcpy.CopyRows_management(out_table, db_connection + "\\" + table_name)

# Print confirmation
print("Accuracyelevation table pushed to PostGIS.")


Accuracyelevation table pushed to PostGIS.


### Raster to point and sending it to postgis

In [10]:
### IDW
import arcpy

# Step 1: Downsample Raster
def downsample_raster(input_raster, output_raster, cell_size):
    arcpy.Resample_management(input_raster, output_raster, cell_size)

# Step 2: Convert Raster to Points
def raster_to_points(input_raster, output_points):
    arcpy.RasterToPoint_conversion(input_raster, output_points, "VALUE")

# Step 3: Upload Points to SDE
def upload_points_to_sde(input_points, output_sde_connection, output_sde_feature_class):
    arcpy.FeatureClassToFeatureClass_conversion(input_points, output_sde_connection, output_sde_feature_class)

# Paths and parameters
input_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb\Idw_ele'
output_downsampled_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\downsampled_idwele.tif'
output_points = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\\idwele.shp'
output_sde_connection = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\PostgreSQL-34-gis5572(postgres).sde'
output_sde_feature_class = 'idwpoints_in_sde'

# Step 1: Downsample Raster
downsample_raster(input_raster, output_downsampled_raster, "15000")

# Step 2: Convert Raster to Points
raster_to_points(output_downsampled_raster, output_points)

# Step 3: Upload Points to SDE
upload_points_to_sde(output_points, output_sde_connection, output_sde_feature_class)


In [12]:
## Ordinaru krigging
# Step 1: Downsample Raster
def downsample_raster(input_raster, output_raster, cell_size):
    arcpy.Resample_management(input_raster, output_raster, cell_size)

# Step 2: Convert Raster to Points
def raster_to_points(input_raster, output_points):
    arcpy.RasterToPoint_conversion(input_raster, output_points, "VALUE")

# Step 3: Upload Points to SDE
def upload_points_to_sde(input_points, output_sde_connection, output_sde_feature_class):
    arcpy.FeatureClassToFeatureClass_conversion(input_points, output_sde_connection, output_sde_feature_class)

# Paths and parameters
input_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb\Kriging_ele'
output_downsampled_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\downsampled_krigele.tif'
output_points = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\\krigele.shp'
output_sde_connection = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\PostgreSQL-34-gis5572(postgres).sde'
output_sde_feature_class = 'krigpoints'

# Step 1: Downsample Raster
downsample_raster(input_raster, output_downsampled_raster, "15000")

# Step 2: Convert Raster to Points
raster_to_points(output_downsampled_raster, output_points)

# Step 3: Upload Points to SDE
upload_points_to_sde(output_points, output_sde_connection, output_sde_feature_class)


In [13]:
#Universal Krigging
# Step 1: Downsample Raster
def downsample_raster(input_raster, output_raster, cell_size):
    arcpy.Resample_management(input_raster, output_raster, cell_size)

# Step 2: Convert Raster to Points
def raster_to_points(input_raster, output_points):
    arcpy.RasterToPoint_conversion(input_raster, output_points, "VALUE")

# Step 3: Upload Points to SDE
def upload_points_to_sde(input_points, output_sde_connection, output_sde_feature_class):
    arcpy.FeatureClassToFeatureClass_conversion(input_points, output_sde_connection, output_sde_feature_class)

# Paths and parameters
input_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\Lab3elevation.gdb\Kriging_uniele'
output_downsampled_raster = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\downsampled_kriguniele.tif'
output_points = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab3elevation\\kriguniele.shp'
output_sde_connection = r'C:\Users\samik\OneDrive\Documents\ArcGIS\Projects\Lab 2\PostgreSQL-34-gis5572(postgres).sde'
output_sde_feature_class = 'krigunipoints'

# Step 1: Downsample Raster
downsample_raster(input_raster, output_downsampled_raster, "15000")

# Step 2: Convert Raster to Points
raster_to_points(output_downsampled_raster, output_points)

# Step 3: Upload Points to SDE
upload_points_to_sde(output_points, output_sde_connection, output_sde_feature_class)
